In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

DSBA

In [2]:
DSBA_students_data = pd.read_excel("data_v1.xlsx")
DSBA_students_data = DSBA_students_data[DSBA_students_data['COURSE_DESCRIPTION'] != 'MSc in Software Engineering']
DSBA_students_data = DSBA_students_data.drop(["INTAKE_CODE", "MODULE_CODE", "STUDY_MODE"], axis=1)

In [3]:
module_name_changes = {
    'Big Data Analytics and Technologies': 'BDAT',
    'Data Management': 'DM',
    'Business Intelligence Systems': 'BIS',
    'Applied Machine Learning': 'AML',
    'Applied Machine Learning ': 'AML',
    'Research Methods for Capstone Project': 'RMCP',
    'Research Methodology for Capstone Project': 'RMCP',
    'Research Methodology in Computing and Engineering': 'RMCP',
    'Research Methodology': 'RMCP',
    'Research Methods for Capstone Project': 'RMCP',
    'Multivariate Methods for Data Analysis': 'MMDA',
    'Data Analytical Programming': 'DAP',
    'Advanced Business Analytics and Visualization': 'ABAV',
    'Advanced Business Analytics and Visualisation': 'ABAV',
    'Behavioural Science, Social Media and Marketing Analytics': 'BSSMMA',
    'Behavioural Science,Social Media and Marketing Analytics': 'BSSMMA',
    'Cloud Infrastructure and Services': 'CIS',
    'Time Series Analysis and Forecasting': 'TSF',
    'Time Series Analysis and Forecasting ': 'TSF',
    'Deep Learning': 'DL',
    'Multilevel Data Analysis': 'MDA',
    'Natural Language Processing': 'NLP',
    'Strategies in Emerging Markets': 'SEM',
    'Data Protection and Management': 'DPM',
    'Operational Research and Optimisation': 'ORO',
    'Operational Research Optimisation': 'ORO',
    'Building IoT Applications': 'BIA',
}

DSBA_students_data['MODULE_NAME'] = DSBA_students_data['MODULE_NAME'].replace(module_name_changes)
DSBA_students_data['MODULE_NAME'].unique()

array(['MMDA', 'TSF', 'MDA', 'ORO', 'SEM', 'BSSMMA', 'Capstone Project',
       'ABAV', 'AML', 'BDAT', 'BIS', 'DAP', 'DM', 'NLP',
       'Capstone Project 1', 'Capstone Project 2', 'RMCP',
       'Postgraduate Information', 'Statistics',
       'Database for Data Science', 'Programming In Python',
       'Introduction to R Programming', 'DL', 'CIS', 'BIA', 'DPM',
       'Software Design and Development', 'System Development Methods',
       'Strategies in Emerging Markets ', 'Programming in Python',
       'Databases for Data Science', 'Managerial Finance',
       'Statistical Decision Making', 'Organisational Behaviour',
       'Global Strategic Management', 'Strategic Marketing Management',
       'Human Resource Management', 'Project',
       'Managing Creativity and Innovation', 'Entrepreneurship',
       'Technology Management'], dtype=object)

In [4]:
valid_modules = ['BDAT', 'DM', 'BIS', 'AML', 'RMCP', 'MMDA', 'DAP', 'ABAV', 'BSSMMA', 'CIS',
                        'TSF', 'DL', 'MDA', 'NLP', 'SEM', 'DPM', 'ORO', 'BIA']
DSBA_students_data = DSBA_students_data[DSBA_students_data['MODULE_NAME'].isin(valid_modules)]

In [5]:
DSBA_students_data['MODULE_NAME'].unique()

array(['MMDA', 'TSF', 'MDA', 'ORO', 'SEM', 'BSSMMA', 'ABAV', 'AML',
       'BDAT', 'BIS', 'DAP', 'DM', 'NLP', 'RMCP', 'DL', 'CIS', 'BIA',
       'DPM'], dtype=object)

In [6]:
duplicated_modules = DSBA_students_data[DSBA_students_data.duplicated(['STUDENT_NUMBER', 'MODULE_NAME'], keep=False)]
duplicated_modules

,STUDENT_NUMBER,COURSE_DESCRIPTION,MODULE_NAME,GRADE
40,ABC0003,MSc in Data Science and Business Analytics,BIS,NaN
41,ABC0003,MSc in Data Science and Business Analytics,DAP,NaN
42,ABC0003,MSc in Data Science and Business Analytics,DM,NaN
46,ABC0003,MSc in Data Science and Business Analytics,RMCP,NaN
48,ABC0003,MSc in Data Science and Business Analytics,DL,NaN
...,...,...,...,...
6133,ABC0054,MSc in Data Science and Business Analytics,AML,NaN
6134,ABC0054,MSc in Data Science and Business Analytics,BDAT,B+
6135,ABC0054,MSc in Data Science and Business Analytics,BIS,NaN
6136,ABC0054,MSc in Data Science and Business Analytics,DAP,NaN


In [7]:
duplicated_modules = DSBA_students_data[DSBA_students_data.duplicated(['STUDENT_NUMBER', 'MODULE_NAME'], keep=False)]

# Function to decide which row to keep based on 'GRADE'
def filter_grades(group):
    if group['GRADE'].notna().sum() > 0:
        return group[group['GRADE'].notna()].drop_duplicates(subset=['STUDENT_NUMBER', 'MODULE_NAME'], keep='first')
    else:
        return group.drop_duplicates(subset=['STUDENT_NUMBER', 'MODULE_NAME'], keep='first')

# Apply the function to each group
filtered_students = duplicated_modules.groupby(['STUDENT_NUMBER', 'MODULE_NAME']).apply(filter_grades).reset_index(drop=True)

# Combine with original DataFrame to get the final cleaned DataFrame
DSBA_students_data = pd.concat([DSBA_students_data.drop(index=duplicated_modules.index), filtered_students])

In [8]:
# Function to check if all 'BSSMMA', 'TSF', and 'CIS' exist in MODULE_NAME for a student
def has_modules(group):
    return {"BSSMMA", "TSF", "CIS"}.issubset(group['MODULE_NAME'].unique())

students_with_both_modules = DSBA_students_data.groupby('STUDENT_NUMBER').filter(has_modules)
students_with_both = students_with_both_modules['STUDENT_NUMBER'].unique()

if len(students_with_both) > 0:
    print("Students with all 'BSSMMA', 'TSF' and 'CIS':")
    print(students_with_both)
else:
    print("No students have all 'BSSMMA', 'TSF' and 'CIS'.")

No students have all 'BSSMMA', 'TSF' and 'CIS'.


In [ ]:
# Function to check if all 'BSSMMA', 'TSF', and 'DL' exist in MODULE_NAME for a student
def has_modules(group):
    return {"BSSMMA", "TSF", "DL"}.issubset(group['MODULE_NAME'].unique())

students_with_all = DSBA_students_data.groupby('STUDENT_NUMBER').filter(has_modules)
students_with_all_numbers = students_with_all['STUDENT_NUMBER'].unique()

rows_to_drop = (DSBA_students_data['STUDENT_NUMBER'].isin(students_with_all_numbers)) & \
               (DSBA_students_data['MODULE_NAME'] == 'DL')

# Drop only those specific rows
DSBA_students_data = DSBA_students_data[~rows_to_drop]

if len(students_with_all_numbers) > 0:
    print(f"Found {len(students_with_all_numbers)} students with all 'BSSMMA', 'TSF' and 'DL'")
    print("Dropped 'DL' entries for these students while keeping their other modules")
    print(students_with_all_numbers)
else:
    print("No students have all three modules")

Found 2 students with all 'BSSMMA', 'TSF' and 'DL'
Dropped 'DL' entries for these students while keeping their other modules
['ABC0129' 'ABC0539']


In [10]:
# Function to check if all 'TSF', 'CIS' and 'DL' exist in MODULE_NAME for a student
def has_modules(group):
    return {"TSF", "CIS", "DL"}.issubset(group['MODULE_NAME'].unique())

students_with_all = DSBA_students_data.groupby('STUDENT_NUMBER').filter(has_modules)
students_with_all_numbers = students_with_all['STUDENT_NUMBER'].unique()

rows_to_drop = (DSBA_students_data['STUDENT_NUMBER'].isin(students_with_all_numbers)) & \
               (DSBA_students_data['MODULE_NAME'] == 'TSF')

# Drop only those specific rows
DSBA_students_data = DSBA_students_data[~rows_to_drop]

if len(students_with_all_numbers) > 0:
    print(f"Found {len(students_with_all_numbers)} students with all 'TSF', 'CIS' and 'DL'")
    print("Dropped 'TSF' entries for these students while keeping their other modules")
    print(students_with_all_numbers)
else:
    print("No students have all three modules")

Found 2 students with all 'TSF', 'CIS' and 'DL'
Dropped 'TSF' entries for these students while keeping their other modules
['ABC0072' 'ABC0073']


In [11]:
# Function to check if all 'BSSMMA', 'CIS' and 'DL' exist in MODULE_NAME for a student
def has_modules(group):
    return {"BSSMMA", "CIS", "DL"}.issubset(group['MODULE_NAME'].unique())

students_with_all = DSBA_students_data.groupby('STUDENT_NUMBER').filter(has_modules)
students_with_all_numbers = students_with_all['STUDENT_NUMBER'].unique()

rows_to_drop = (DSBA_students_data['STUDENT_NUMBER'].isin(students_with_all_numbers)) & \
               (DSBA_students_data['MODULE_NAME'] == 'BSSMMA')

# Drop only those specific rows
DSBA_students_data = DSBA_students_data[~rows_to_drop]

if len(students_with_all_numbers) > 0:
    print(f"Found {len(students_with_all_numbers)} students with all 'BSSMMA', 'CIS' and 'DL'")
    print("Dropped 'BSSMMA' entries for these students while keeping their other modules")
    print(students_with_all_numbers)
else:
    print("No students have all three modules")

Found 2 students with all 'BSSMMA', 'CIS' and 'DL'
Dropped 'BSSMMA' entries for these students while keeping their other modules
['ABC0173' 'ABC0083']


In [30]:
# Function to check DE elective in BI pathway
def check_modules(group):
    has_both = {"CIS", "DL"}.issubset(group['MODULE_NAME'].unique())
    has_any_of_three = {'MDA', 'SEM', 'ORO'}.intersection(group['MODULE_NAME'].unique())
    return has_both and bool(has_any_of_three)

students_with_criteria = DSBA_students_data.groupby('STUDENT_NUMBER').filter(check_modules)
students_matching_criteria = students_with_criteria['STUDENT_NUMBER'].unique()

if len(students_matching_criteria) > 0:
    print("Students matching the criteria (both 'BSSMMA' and 'TSF', and at least one of 'MDA', 'SEM', 'ORO'):")
    print(students_matching_criteria)
else:
    print("No students match the criteria.")

No students match the criteria.


In [12]:
# Function to check  BI elective in DE pathway
def check_modules(group):
    has_both = {"BSSMMA", "TSF"}.issubset(group['MODULE_NAME'].unique())
    has_any_of_three = {"NLP", "DPM", "BIA"}.intersection(group['MODULE_NAME'].unique())
    return has_both and bool(has_any_of_three)

students_with_criteria = DSBA_students_data.groupby('STUDENT_NUMBER').filter(check_modules)
students_matching_criteria = students_with_criteria['STUDENT_NUMBER'].unique()

if len(students_matching_criteria) > 0:
    print("Students matching the criteria (both 'BSSMMA' and 'TSF', and at least one of 'NLP', 'DPM', 'BIA'):")
    print(students_matching_criteria)
else:
    print("No students match the criteria.")

Students matching the criteria (both 'BSSMMA' and 'TSF', and at least one of 'NLP', 'DPM', 'BIA'):
['ABC0001' 'ABC0005' 'ABC0007' 'ABC0009' 'ABC0010' 'ABC0011' 'ABC0012'
 'ABC0014' 'ABC0015' 'ABC0017' 'ABC0101' 'ABC0114' 'ABC0115' 'ABC0117'
 'ABC0121' 'ABC0262' 'ABC0263' 'ABC0264' 'ABC0265' 'ABC0267' 'ABC0268'
 'ABC0259' 'ABC0260' 'ABC0261' 'ABC0270' 'ABC0272' 'ABC0273' 'ABC0275'
 'ABC0276' 'ABC0278' 'ABC0269' 'ABC0280' 'ABC0281' 'ABC0283' 'ABC0284'
 'ABC0285' 'ABC0286' 'ABC0287' 'ABC0279' 'ABC0289' 'ABC0290' 'ABC0292'
 'ABC0293' 'ABC0294' 'ABC0295' 'ABC0288' 'ABC0557' 'ABC0502' 'ABC0504'
 'ABC0505' 'ABC0506' 'ABC0507' 'ABC0518' 'ABC0521' 'ABC0522' 'ABC0123'
 'ABC0124' 'ABC0127' 'ABC0128' 'ABC0129' 'ABC0130' 'ABC0524' 'ABC0532'
 'ABC0533' 'ABC0538' 'ABC0539' 'ABC0540' 'ABC0544' 'ABC0550' 'ABC0131']


In [13]:
def filter_rows(group):
    has_both = {"BSSMMA", "TSF"}.issubset(group['MODULE_NAME'].unique())
    if has_both:
        # Drop rows where MODULE_NAME is in {"NLP", "DPM", "BIA"}
        return group[~group['MODULE_NAME'].isin({"NLP", "DPM", "BIA"})]
    else:
        return group

DSBA_students_data = DSBA_students_data.groupby('STUDENT_NUMBER', group_keys=False).apply(filter_rows)

In [14]:
DSBA_students_data[(DSBA_students_data['STUDENT_NUMBER']=='ABC0001')]

,STUDENT_NUMBER,COURSE_DESCRIPTION,MODULE_NAME,GRADE
0,ABC0001,MSc in Data Science and Business Analytics,MMDA,B+
1,ABC0001,MSc in Data Science and Business Analytics,TSF,C-
2,ABC0001,MSc in Data Science and Business Analytics,MDA,A
3,ABC0001,MSc in Data Science and Business Analytics,ORO,NaN
4,ABC0001,MSc in Data Science and Business Analytics,SEM,NaN
5,ABC0001,MSc in Data Science and Business Analytics,BSSMMA,A
7,ABC0001,MSc in Data Science and Business Analytics,ABAV,B+
8,ABC0001,MSc in Data Science and Business Analytics,AML,C-
9,ABC0001,MSc in Data Science and Business Analytics,BDAT,C+
10,ABC0001,MSc in Data Science and Business Analytics,BIS,B+


In [15]:
DSBA_students_data.reset_index(drop=True, inplace=True)

In [16]:
module_name_changes = {
    'RMCP': 'RMCE | RMCP',
    'NLP': 'MDA | NLP',
    'MDA': 'MDA | NLP',
    'BSSMMA': 'BSSMMA | CIS',
    'CIS': 'BSSMMA | CIS',
    'TSF': 'TSF | DL',
    'DL': 'TSF | DL',
    'SEM': 'SEM | DPM',
    'DPM': 'SEM | DPM',
    'ORO': 'ORO | BIA',
    'BIA': 'ORO | BIA'
}

DSBA_students_data['MODULE_NAME'] = DSBA_students_data['MODULE_NAME'].replace(module_name_changes)
DSBA_students_data['MODULE_NAME'].unique()

array(['MMDA', 'TSF | DL', 'MDA | NLP', 'ORO | BIA', 'SEM | DPM',
       'BSSMMA | CIS', 'ABAV', 'AML', 'BDAT', 'BIS', 'DAP', 'DM',
       'RMCE | RMCP'], dtype=object)

In [17]:
DSBA_students_data.head()

,STUDENT_NUMBER,COURSE_DESCRIPTION,MODULE_NAME,GRADE
0,ABC0001,MSc in Data Science and Business Analytics,MMDA,B+
1,ABC0001,MSc in Data Science and Business Analytics,TSF | DL,C-
2,ABC0001,MSc in Data Science and Business Analytics,MDA | NLP,A
3,ABC0001,MSc in Data Science and Business Analytics,ORO | BIA,NaN
4,ABC0001,MSc in Data Science and Business Analytics,SEM | DPM,NaN


In [18]:
duplicated_modules = DSBA_students_data[DSBA_students_data.duplicated(['STUDENT_NUMBER', 'MODULE_NAME'], keep=False)]
print(duplicated_modules)

Empty DataFrame
Columns: [STUDENT_NUMBER, COURSE_DESCRIPTION, MODULE_NAME, GRADE]
Index: []


In [19]:
DSBA_each_student_info = DSBA_students_data.groupby('STUDENT_NUMBER').apply(
    lambda x:{'Modules': x['MODULE_NAME'].tolist(), 'Grades': x['GRADE'].tolist()}
).to_dict()

print(DSBA_each_student_info)

{'ABC0001': {'Modules': ['MMDA', 'TSF | DL', 'MDA | NLP', 'ORO | BIA', 'SEM | DPM', 'BSSMMA | CIS', 'ABAV', 'AML', 'BDAT', 'BIS', 'DAP', 'DM', 'RMCE | RMCP'], 'Grades': ['B+', 'C-', 'A', nan, nan, 'A', 'B+', 'C-', 'C+', 'B+', 'A', 'B+', 'A']}, 'ABC0002': {'Modules': ['MMDA', 'TSF | DL', 'MDA | NLP', 'ORO | BIA', 'SEM | DPM', 'BSSMMA | CIS', 'ABAV', 'AML', 'BDAT', 'BIS', 'DAP', 'DM', 'RMCE | RMCP'], 'Grades': ['A', 'C-', 'C-', nan, nan, 'A', 'B', 'B', 'A', 'C+', 'B+', 'B+', 'B']}, 'ABC0003': {'Modules': ['MMDA', 'ABAV', 'AML', 'BDAT', 'BIS', 'BSSMMA | CIS', 'DAP', 'TSF | DL', 'DM', 'RMCE | RMCP'], 'Grades': ['C+', 'B+', 'B+', 'B+', 'A', 'C+', 'B+', 'C+', nan, 'B+']}, 'ABC0004': {'Modules': ['MMDA', 'ABAV', 'AML', 'BDAT', 'BIS', 'DAP', 'DM', 'MDA | NLP', 'RMCE | RMCP', 'TSF | DL', 'BSSMMA | CIS', 'ORO | BIA', 'SEM | DPM'], 'Grades': ['A', 'B+', 'B', 'A', 'A+', 'B+', 'A', 'A', 'A+', 'B+', 'B+', nan, nan]}, 'ABC0005': {'Modules': ['MMDA', 'TSF | DL', 'MDA | NLP', 'ORO | BIA', 'SEM | DPM', 

In [20]:
incomplete_modules = {}
completed_modules = {}

for student_id, modules_info in DSBA_each_student_info.items():
    modules  = modules_info['Modules']
    grades = modules_info['Grades']

    incomplete_modules[student_id] = []
    completed_modules[student_id] = []
    
    for module, grade in zip(modules, grades):
        if pd.isna(grade):
            incomplete_modules[student_id].append(module)
        else:
            completed_modules[student_id].append(module)
            
print("Incomplete Modules:", incomplete_modules)
print("Completed Modules:", completed_modules)

Incomplete Modules: {'ABC0001': ['ORO | BIA', 'SEM | DPM'], 'ABC0002': ['ORO | BIA', 'SEM | DPM'], 'ABC0003': ['DM'], 'ABC0004': ['ORO | BIA', 'SEM | DPM'], 'ABC0005': ['MMDA', 'TSF | DL', 'MDA | NLP', 'ORO | BIA', 'SEM | DPM', 'ABAV', 'AML', 'BDAT', 'BIS', 'RMCE | RMCP'], 'ABC0006': ['MDA | NLP', 'SEM | DPM'], 'ABC0007': ['SEM | DPM'], 'ABC0008': ['MDA | NLP', 'SEM | DPM'], 'ABC0009': ['SEM | DPM'], 'ABC0010': ['MDA | NLP', 'SEM | DPM'], 'ABC0011': ['TSF | DL', 'ORO | BIA', 'SEM | DPM'], 'ABC0012': ['SEM | DPM'], 'ABC0014': ['ORO | BIA'], 'ABC0015': ['TSF | DL', 'SEM | DPM'], 'ABC0016': ['ORO | BIA', 'SEM | DPM'], 'ABC0017': ['TSF | DL', 'SEM | DPM'], 'ABC0018': ['ORO | BIA', 'SEM | DPM'], 'ABC0019': ['MMDA', 'TSF | DL', 'MDA | NLP', 'ORO | BIA', 'SEM | DPM', 'BSSMMA | CIS', 'BIS', 'DAP', 'RMCE | RMCP'], 'ABC0020': ['MDA | NLP', 'ORO | BIA', 'SEM | DPM'], 'ABC0021': ['ORO | BIA'], 'ABC0022': ['MMDA', 'TSF | DL', 'BSSMMA | CIS', 'ABAV', 'AML', 'BDAT', 'BIS', 'DAP', 'DM', 'RMCE | RMCP']

In [21]:
for student_id, module in incomplete_modules.items():
    print(student_id, module)

ABC0001 ['ORO | BIA', 'SEM | DPM']
ABC0002 ['ORO | BIA', 'SEM | DPM']
ABC0003 ['DM']
ABC0004 ['ORO | BIA', 'SEM | DPM']
ABC0005 ['MMDA', 'TSF | DL', 'MDA | NLP', 'ORO | BIA', 'SEM | DPM', 'ABAV', 'AML', 'BDAT', 'BIS', 'RMCE | RMCP']
ABC0006 ['MDA | NLP', 'SEM | DPM']
ABC0007 ['SEM | DPM']
ABC0008 ['MDA | NLP', 'SEM | DPM']
ABC0009 ['SEM | DPM']
ABC0010 ['MDA | NLP', 'SEM | DPM']
ABC0011 ['TSF | DL', 'ORO | BIA', 'SEM | DPM']
ABC0012 ['SEM | DPM']
ABC0014 ['ORO | BIA']
ABC0015 ['TSF | DL', 'SEM | DPM']
ABC0016 ['ORO | BIA', 'SEM | DPM']
ABC0017 ['TSF | DL', 'SEM | DPM']
ABC0018 ['ORO | BIA', 'SEM | DPM']
ABC0019 ['MMDA', 'TSF | DL', 'MDA | NLP', 'ORO | BIA', 'SEM | DPM', 'BSSMMA | CIS', 'BIS', 'DAP', 'RMCE | RMCP']
ABC0020 ['MDA | NLP', 'ORO | BIA', 'SEM | DPM']
ABC0021 ['ORO | BIA']
ABC0022 ['MMDA', 'TSF | DL', 'BSSMMA | CIS', 'ABAV', 'AML', 'BDAT', 'BIS', 'DAP', 'DM', 'RMCE | RMCP']
ABC0023 ['MDA | NLP', 'SEM | DPM']
ABC0024 ['MMDA', 'BIS', 'DAP', 'DM', 'MDA | NLP', 'RMCE | RMCP', 'TS

In [22]:
for student_id, modules in incomplete_modules.items():
    if len(modules) != len(set(modules)):
        print(f"Student {student_id} has duplicate modules: {modules}")

In [23]:
elective = ["MDA | NLP", "SEM | DPM", "ORO | BIA"]
for student_id, completed_modules_info in completed_modules.items():
    completed_elective = set(completed_modules_info) & set(elective)
    
    if len(completed_elective) >= 1:
        for student_id, incomplete_modules_info in incomplete_modules.items():
            extra_elective = set(incomplete_modules_info) & set(elective)

            for module in extra_elective:
                completed_modules[student_id].append(module)
                incomplete_modules_info.remove(module)
                
print("Incomplete Modules:", incomplete_modules)
print("Completed Modules:", completed_modules)

Incomplete Modules: {'ABC0001': [], 'ABC0002': [], 'ABC0003': ['DM'], 'ABC0004': [], 'ABC0005': ['MMDA', 'TSF | DL', 'ABAV', 'AML', 'BDAT', 'BIS', 'RMCE | RMCP'], 'ABC0006': [], 'ABC0007': [], 'ABC0008': [], 'ABC0009': [], 'ABC0010': [], 'ABC0011': ['TSF | DL'], 'ABC0012': [], 'ABC0014': [], 'ABC0015': ['TSF | DL'], 'ABC0016': [], 'ABC0017': ['TSF | DL'], 'ABC0018': [], 'ABC0019': ['MMDA', 'TSF | DL', 'BSSMMA | CIS', 'BIS', 'DAP', 'RMCE | RMCP'], 'ABC0020': [], 'ABC0021': [], 'ABC0022': ['MMDA', 'TSF | DL', 'BSSMMA | CIS', 'ABAV', 'AML', 'BDAT', 'BIS', 'DAP', 'DM', 'RMCE | RMCP'], 'ABC0023': [], 'ABC0024': ['MMDA', 'BIS', 'DAP', 'DM', 'RMCE | RMCP', 'TSF | DL', 'BSSMMA | CIS'], 'ABC0025': [], 'ABC0026': [], 'ABC0027': ['RMCE | RMCP'], 'ABC0028': [], 'ABC0029': [], 'ABC0030': ['MMDA', 'TSF | DL', 'BSSMMA | CIS', 'ABAV', 'AML', 'BDAT', 'BIS', 'RMCE | RMCP', 'DAP', 'DM'], 'ABC0031': ['MMDA', 'TSF | DL', 'BSSMMA | CIS', 'ABAV', 'AML', 'BDAT', 'BIS', 'RMCE | RMCP', 'DAP', 'DM'], 'ABC0032': 

In [24]:
for student_id, modules in incomplete_modules.items():
    if len(modules) != len(set(modules)):
        print(f"Student {student_id} has duplicate modules: {modules}")